# Active - Passive AllenNLP SRL BiLSTM

In [1]:
from allennlp_models.pretrained import load_predictor

/home/elena/anaconda3/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


In [2]:
import csv

In [3]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [4]:
from checklist.pred_wrapper import PredictorWrapper

In [5]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

## Active

In [6]:
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Peter cuddled someone.")
output

{'verbs': [{'verb': 'cuddled',
   'description': '[ARG0: Peter] [V: cuddled] [ARG1: someone] .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
 'words': ['Peter', 'cuddled', 'someone', '.']}

In [7]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [9]:
d = ["Peter cuddled someone."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'cuddled',
     'description': '[ARG0: Peter] [V: cuddled] [ARG1: someone] .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'O']}],
   'words': ['Peter', 'cuddled', 'someone', '.']}],
 array([1.]))

In [10]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [11]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [12]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [13]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [14]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [12]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} cuddled someone.", meta=True,nsamples=50, remove_duplicates=True)
with open('/datasets/active_arg0_cuddled_bilstm.txt', 'w') as f:
    print(t.data, file = f)

import csv
# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_cuddled_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    24 (48.0%)

Example fails:
Jerry [V: cuddled] [ARG1: someone .]
----
Roy [V: cuddled] [ARG1: someone] .
----
[ARGM-MNR: Patrick] [V: cuddled] [ARG1: someone] .
----
['Larry', 'cuddled', 'someone', '.'] [False]
['Nick', 'cuddled', 'someone', '.'] [False]
['Mark', 'cuddled', 'someone', '.'] [ True]
['Mark', 'cuddled', 'someone', '.'] [ True]
['Joe', 'cuddled', 'someone', '.'] [False]
['Ed', 'cuddled', 'someone', '.'] [False]
['Edwin', 'cuddled', 'someone', '.'] [ True]
['Sandra', 'cuddled', 'someone', '.'] [ True]
['Laura', 'cuddled', 'someone', '.'] [False]
['Sue', 'cuddled', 'someone', '.'] [ True]
['Susan', 'cuddled', 'someone', '.'] [False]
['Kate', 'cuddled', 'someone', '.'] [False]
['Tim', 'cuddled', 'someone', '.'] [False]
['Rebecca', 'cuddled', 'someone', '.'] [False]
['Lynn', 'cuddled', 'someone', '.'] [ True]
['Kathy', 'cuddled', 'someone', '.'] [False]
['Katherine', 'cuddled', 'someone', '.'] [ True]
['Lawrence', 'cud

In [15]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("Peter cuddled {first_name}.", meta=True,nsamples=50, remove_duplicates=True)
with open('/datasets/active_arg1_cuddled_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg1_cuddled_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    14 (28.0%)

Example fails:
Peter [V: cuddled] [ARG2: Deborah] .
----
[ARG0: Peter] [V: cuddled] [ARG2: Billy] .
----
Peter [V: cuddled] [ARG2: Ellen] .
----
['Peter', 'cuddled', 'Donna', '.'] [ True]
['Peter', 'cuddled', 'Louis', '.'] [ True]
['Peter', 'cuddled', 'Charlotte', '.'] [ True]
['Peter', 'cuddled', 'Anne', '.'] [False]
['Peter', 'cuddled', 'Sandra', '.'] [False]
['Peter', 'cuddled', 'Pamela', '.'] [ True]
['Peter', 'cuddled', 'Fiona', '.'] [ True]
['Peter', 'cuddled', 'Anna', '.'] [ True]
['Peter', 'cuddled', 'Joseph', '.'] [False]
['Peter', 'cuddled', 'Diana', '.'] [False]
['Peter', 'cuddled', 'Jimmy', '.'] [ True]
['Peter', 'cuddled', 'Jessica', '.'] [ True]
['Peter', 'cuddled', 'Tim', '.'] [ True]
['Peter', 'cuddled', 'Benjamin', '.'] [ True]
['Peter', 'cuddled', 'Sophie', '.'] [ True]
['Peter', 'cuddled', 'Howard', '.'] [ True]
['Peter', 'cuddled', 'Nick', '.'] [False]
['Peter', 'cuddled', 'Jeff', '.'] [ True]
[

In [13]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} kissed someone.", meta=True,nsamples=50, remove_duplicates=True)

with open('/datasets/active_arg0_kissed_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_kissed_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    2 (4.0%)

Example fails:
Anthony [V: kissed] [ARG1: someone] .
----
[ARGM-ADV: Alfred] [V: kissed] [ARG1: someone] .
----
['Stephanie', 'kissed', 'someone', '.'] [ True]
['Christopher', 'kissed', 'someone', '.'] [ True]
['Marilyn', 'kissed', 'someone', '.'] [ True]
['Robin', 'kissed', 'someone', '.'] [ True]
['Sally', 'kissed', 'someone', '.'] [ True]
['Ray', 'kissed', 'someone', '.'] [ True]
['Jay', 'kissed', 'someone', '.'] [ True]
['Katherine', 'kissed', 'someone', '.'] [ True]
['Christopher', 'kissed', 'someone', '.'] [ True]
['Ryan', 'kissed', 'someone', '.'] [ True]
['Christine', 'kissed', 'someone', '.'] [ True]
['Lisa', 'kissed', 'someone', '.'] [ True]
['Sandra', 'kissed', 'someone', '.'] [ True]
['Alexander', 'kissed', 'someone', '.'] [ True]
['Alison', 'kissed', 'someone', '.'] [ True]
['Evelyn', 'kissed', 'someone', '.'] [ True]
['Anthony', 'kissed', 'someone', '.'] [False]
['Tom', 'kissed', 'someone', '.'] [ True]

# Negation

In [8]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description']

In [9]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [10]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']
        
    return predicate_structure

In [11]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [12]:
d = ["Peter doesn't cuddled someone."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'does',
     'description': "Peter [V: does] n't cuddled someone .",
     'tags': ['O', 'B-V', 'O', 'O', 'O', 'O']},
    {'verb': 'cuddled',
     'description': "[ARG0: Peter] does [ARGM-NEG: n't] [V: cuddled] [ARG1: someone] .",
     'tags': ['B-ARG0', 'O', 'B-ARGM-NEG', 'B-V', 'B-ARG1', 'O']}],
   'words': ['Peter', 'does', "n't", 'cuddled', 'someone', '.']}],
 array([1.]))

In [20]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} does not cuddle someone.", meta=True,nsamples=50, remove_duplicates=True)

with open('/datasets/active_arg0_negation1_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_negation1_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    0 (0.0%)
['John', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Elizabeth', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Elizabeth', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Jean', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Maria', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Elizabeth', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Hugh', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Steven', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Ron', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Ruth', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Judith', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Sandra', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Chris', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Alan', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Charles', 'does', 'not', 'cuddle', 'someone', '.'] [ True]
['Henry', 'does', 'not', 'cuddle', '

In [21]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} doesn't cuddle someone.", meta=True,nsamples=50, remove_duplicates=True)

with open('/datasets/active_arg0_negation1.2_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_negation1.2_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    0 (0.0%)
['Ruth', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Heather', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Ed', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Adam', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Sam', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Ed', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Helen', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Angela', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Harry', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Jeff', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Alfred', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Diane', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Simon', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Jim', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Fiona', 'does', "n't", 'cuddle', 'someone', '.'] [ True]
['Marie', 'does', "n't", 'cuddle', 'someone', '.'] [ Tr

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [16]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [17]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [18]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [20]:
# initialize editor object
editor = Editor()

# create examples ARG0
t = editor.template("{first_name} cuddled no one.", meta=True,nsamples=50, remove_duplicates=True)

with open('/datasets/active_arg0_negation2_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/active_arg0_negation2_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 50 examples
Test cases:      50
Fails (rate):    17 (34.0%)

Example fails:
[ARGM-MNR: Louis] [V: cuddled] [ARG1: no one] .
----
[ARGM-MNR: Ken] [V: cuddled] [ARG1: no one] .
----
[ARGM-MNR: Victoria] [V: cuddled] [ARG1: no one] .
----
['Claire', 'cuddled', 'no', 'one', '.'] [ True]
['Steve', 'cuddled', 'no', 'one', '.'] [ True]
['Edward', 'cuddled', 'no', 'one', '.'] [ True]
['Louis', 'cuddled', 'no', 'one', '.'] [False]
['Martha', 'cuddled', 'no', 'one', '.'] [False]
['Billy', 'cuddled', 'no', 'one', '.'] [ True]
['Louise', 'cuddled', 'no', 'one', '.'] [False]
['Philip', 'cuddled', 'no', 'one', '.'] [ True]
['Kevin', 'cuddled', 'no', 'one', '.'] [ True]
['Edward', 'cuddled', 'no', 'one', '.'] [ True]
['Ben', 'cuddled', 'no', 'one', '.'] [False]
['Mark', 'cuddled', 'no', 'one', '.'] [ True]
['Steven', 'cuddled', 'no', 'one', '.'] [ True]
['Kim', 'cuddled', 'no', 'one', '.'] [ True]
['Victoria', 'cuddled', 'no', 'one', '.'] [False]
['Lauren', 'cuddled', 'no', 'one', '.'] [Fa

# End of Notebook